<a href="https://colab.research.google.com/github/HanJaeHui-GitHub/ESAA/blob/main/ESAA_OB_Week5_markdown_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 5주차 과제_0404(금)


파이썬 머신러닝 완벽 가이드 개정 2판 8장 p538~549

## 문서 군집화 소개와 실습(Opinion Review 데이터 세트)

### 문서 군집화 개념

문서 군집화(Document Clustering) : 비슷한 텍스트 구성의 문집를 군집화(Clustering)하는 것

동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류하므로 텍스트 분류 기반의 문서 분류와 유사함

텍스트 분류 기반의 문서 분류는 사전에 결정 카테고리 값을 가진 학습 데이터 세트가 필요한 데 반해 문서 군집화는 학습 데이터 세트가 필요 없는 비지도학습 기반으로 동작

### Opninon Review 데이터 세트를 이용한 문서 군집화 수행하기

In [29]:
import pandas as pd
import glob, os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 700)

path = r'/content/drive/MyDrive/ESAA/25-1 OB 세션/datasets/OpinosisDataset1.0/topics'
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합,
# 개별 파일들의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')

    # 절대경로로 주어진 파일명을 가공. Linux에서 수행 시에는 아래 \를 / 변경
    # 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]

    # 파일명 리스트와 파일 내용 리스트에 파일명과 파일 내용을 추가
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일 내용 리스트를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

filename  \
0      battery-life_ipod_nano_8gb   
1           screen_netbook_1005ha   
2         keyboard_netbook_1005ha   
3     battery-life_netbook_1005ha   
4  satellite_garmin_nuvi_255W_gps   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  opinion_text  
0                                                                                                                                                                                                                                                                                                 short battery life  I moved up from an 8gb .\n0                                                                                                                                                                                                                                                                                            I love this ipod except for the battery life .\n1                             ...  
1                                                                                                                                                                                                                                                                          Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0                                                                                                                                                                                                                                                                                                       I've used mine outsi...  
2                                                                                                                                                                                  ,  I think the new keyboard rivals the great hp mini keyboards .\n0                                                                                                                                 Since the battery life difference is minimum, the only reason to upgrade would be to get the better keyboard .\n1                                                                                                                                                                                   The keyboard is now as good as t...  
3                                                                                                                                                                                                                                                                                                                  6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery  , and a 1 .\n0                                                                                                                                                                                                                                                                                              Not to mention that as of now...  
4                                                                                                                                                                                                                                                                                    

각 파일 이름(filename) 자체만으로 의견(opinion)의 텍스트(text)가 어떠한 제품/서비스에 대한 리뷰인지 알 수 있음

In [30]:
# TF-IDF 형태로 문서 피처 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet', download_dir='/content/drive/MyDrive/ESAA/25-1 OB 세션/datasets/nltk_data')

def LemNormalize(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    return [wordnet_lemmatizer.lemmatize(token) for token in text.split()]

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1,2), min_df=0.05, max_df=0.85)

# opinion_test 칼럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

[nltk_data] Downloading package wordnet to
[nltk_data]     /content/drive/MyDrive/ESAA/25-1 OB
[nltk_data]     세션/datasets/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
# K- 평균으로 군집화를 수행하여 어떤 문서끼리 군집되는지 확인
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state = 0
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [32]:
document_df['cluster_label'] = cluster_label
document_df.head()

filename  \
0      battery-life_ipod_nano_8gb   
1           screen_netbook_1005ha   
2         keyboard_netbook_1005ha   
3     battery-life_netbook_1005ha   
4  satellite_garmin_nuvi_255W_gps   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  opinion_text  \
0                                                                                                                                                                                                                                                                                                 short battery life  I moved up from an 8gb .\n0                                                                                                                                                                                                                                                                                            I love this ipod except for the battery life .\n1                             ...   
1                                                                                                                                                                                                                                                                          Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0                                                                                                                                                                                                                                                                                                       I've used mine outsi...   
2                                                                                                                                                                                  ,  I think the new keyboard rivals the great hp mini keyboards .\n0                                                                                                                                 Since the battery life difference is minimum, the only reason to upgrade would be to get the better keyboard .\n1                                                                                                                                                                                   The keyboard is now as good as t...   
3                                                                                                                                                                                                                                                                                                                  6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery  , and a 1 .\n0                                                                                                                                                                                                                                                                                              Not to mention that as of now...   
4                                                                                                                                                                                                                                                                               

In [33]:
# sort_values(by = 정렬칼럼명) 설정을 통해 정렬
document_df[document_df['cluster_label']==0].sort_values(by ='filename')

filename  \
34     battery-life_amazon_kindle   
0      battery-life_ipod_nano_8gb   
3     battery-life_netbook_1005ha   
14              features_windows7   
42  performance_honda_accord_2008   
27     performance_netbook_1005ha   
11       size_asus_netbook_1005ha   
7      speed_garmin_nuvi_255W_gps   
16                 speed_windows7   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   opinion_text  \
34                                                                                                                                                                                                               After I plugged it in to my USB hub on my computer to charge the battery the charging cord design is very clever !\n0                                                                                                                                     After you have paged tru a 500, page book one, page, at, a, time to get from Chapter 2 to Chapter 15, see how excited you are about a low battery and all the time it took to get there !\n1                                                     ...   
0                                                                                                                                                                                                                                                                                                  short battery life  I moved up from an 8gb .\n0                                                                                                                                                                                                                                                                                            I love this ipod except for the battery life .\n1                             ...   
3                                                                                                                                                                                                                                                                                                                   6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery  , and a 1 .\n0                                                                                                                                                                                                                                                                                              Not to mention that as of now...   
14                                                                                                                                                                             I had to uninstall anti, virus and selected other programs, some of which did not have listings in the  Programs and Features  Control Panel section .\n0                                                                                                                                                                                                                           This review briefly touches upon some of the key features and enhancements of Microsoft's latest OS .\n1                                                 ...   
42                                                                                     Very happy with my 08 Accord, pe

Cluster #0 : 전자기기 및 주요 구성요소에 대한 리뷰로 군집화

In [35]:
document_df[document_df['cluster_label']==1].sort_values(by ='filename')

filename  \
39   bathroom_bestwestern_hotel_sfo   
26          food_holiday_inn_london   
28           food_swissotel_chicago   
41       free_bestwestern_hotel_sfo   
46   location_bestwestern_hotel_sfo   
22      location_holiday_inn_london   
38    parking_bestwestern_hotel_sfo   
29         price_holiday_inn_london   
18          room_holiday_inn_london   
40      rooms_bestwestern_hotel_sfo   
36          rooms_swissotel_chicago   
37    service_bestwestern_hotel_sfo   
21       service_holiday_inn_london   
31  service_swissotel_hotel_chicago   
49      staff_bestwestern_hotel_sfo   
24          staff_swissotel_chicago   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   opinion_text  \
39                                                                                                                                                                                                                                                The room was not overly big, but clean and very comfortable beds, a great shower and very clean bathrooms .\n0                                                                                                                                                                                                                          The second room was smaller, with a very inconvenient bathroom layout, but at least it was quieter and we were able to sleep .\n1 ...   
26                                                                                                                                                                                                                                                                      The room was packed to capacity with queues at the food buffets .\n0                                                                                                                                                   The over zealous   staff cleared our unfinished drinks while we were collecting cooked food and movement around the room with plates was difficult in the crowded circumstances .\n1                                         ...   
28                                                                                                                                                                                                                                                                                                                                                                   The food for our event was delicious .\n0                                                                                                                                                                                                                                                                                                              ...   
41                                                                                                                                                                The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room .\n0                                                                                      They also have a computer available with free internet which is a nice bonus but I didn't find that out till the day before we left but was still

Cluster #1 : 호텔에 대한 리뷰로 군집화

In [36]:
document_df[document_df['cluster_label']==2].sort_values(by ='filename')

filename  \
45       comfort_honda_accord_2008   
32       comfort_toyota_camry_2007   
33   gas_mileage_toyota_camry_2007   
43      interior_honda_accord_2008   
25      interior_toyota_camry_2007   
50       mileage_honda_accord_2008   
47       quality_toyota_camry_2007   
48         seats_honda_accord_2008   
44  transmission_toyota_camry_2007   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   opinion_text  \
45                                                                                                                                                                                                                                                                                                                                                                                                Drivers seat not comfortable, the car itself compared to other models of similar class .\n0                                                                                                                                                                                                                               ...   
32                                                                                                                                                  Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0                                                                                                        Seats are fine, in fact of all the smaller sedans this is the most comfortable I found for the price as I am 6', 2  and 250# .\n1                                                                                                                                                                                     Great gas mileage and comfortable on long trips ...   
33                                                                                                                                                                                                                                                                                                                                                                              Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0                                                                                                                                                                                                                                                  ...   
43                                                                                                                                                                                                                                                                                                                                                                                                                                                                             I love the new body style and the interior is a simple pleasure except for the center dash .\n0                                                                                                                                              ...   
25                                                                                                            

Cluster #2 : 토요타(Toyta)와 혼다(Honda) 등의 자동차에 대한 리뷰로 군집화

In [37]:
document_df[document_df['cluster_label']==3].sort_values(by ='filename')

filename  \
5     accuracy_garmin_nuvi_255W_gps   
17  directions_garmin_nuvi_255W_gps   
4    satellite_garmin_nuvi_255W_gps   
6       screen_garmin_nuvi_255W_gps   
10              sound_ipod_nano_8gb   
8      updates_garmin_nuvi_255W_gps   
12              video_ipod_nano_8gb   
15       voice_garmin_nuvi_255W_gps   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   opinion_text  \
5                                                                                                                                                                                                                                                                  , and is very, very accurate .\n0                                                                                                                                                                           but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1                                                                                                              This functi...   
17                                                                                                                                                                                                                                                                You also get upscale features like spoken directions including street names and programmable POIs .\n0                                                                                                                                                                                                                                                                                                I used to hesitate to go out of my directions but no...   
4                                                                                                                                                                                                                                                                                           It's fast to acquire satellites .\n0    If you've ever had a  Brand X  GPS take you on some strange route that adds 20 minutes to your trip, has you turn the wrong way down a one way road, tell you to turn AFTER you've passed the street, frequently loses the satellite signal, or has old maps missing streets, you know how important this stuff is .\n1                                                                 ...   
6                                                                                                                                                                                                       It is easy to read and when touching the screen it works great !\n0                                                                                                                                                                    and zoom out   buttons on the 255w to the same side of the screen which makes it a bit easier .\n1                                                                                                                                                                           ...   
10     headphone jack i got a clear case for it and it  i got a clear case for it and it like prvents me from being able to put the jack all

Cluster #3 : 차량용 내비게이션 리뷰로 군집화

In [38]:
document_df[document_df['cluster_label']==4].sort_values(by ='filename')

filename  \
19          buttons_amazon_kindle   
9    display_garmin_nuvi_255W_gps   
35  eyesight-issues_amazon_kindle   
30            fonts_amazon_kindle   
2         keyboard_netbook_1005ha   
23       navigation_amazon_kindle   
20            price_amazon_kindle   
13           screen_ipod_nano_8gb   
1           screen_netbook_1005ha   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   opinion_text  \
19                                                                                                                                             I thought it would be fitting to christen my Kindle with the Stephen King novella UR, so went to the Amazon site on my computer and clicked on the button to buy it .\n0                                                                                                                                                                                                            As soon as I'd clicked the button to confirm my order it appeared on my Kindle almost immediately !\n1                                                                                   ...   
9                                                                                                                                                                                                                                                        3 quot  widescreen display was a bonus .\n0                                                                                                                                           This made for smoother graphics on the 255w of the vehicle moving along displayed roads, where the 750's display was more of a  jerky  movement .\n1                                                                                                                         ...   
35                                                                                                                                                                                            It feels as easy to read as the K1 but doesn't seem any crisper to my eyes   .\n0                                                                                                                                                            the white is really GREY, and to avoid considerable eye, strain I had to refresh pages   every other page .\n1                                    The dream has always been a portable electronic device that could hold a ton of reading material, automate subscriptions and fa...   
30                                                                                                                                                                                                                                                                                                                     Being able to change the font sizes is awesome !\n0                                                                                                                                                                                                                                                                        For whatever reason, Amazon decided to make the Font on the Home Screen   ...   
2                                                                                                                      

흠 애매한거 같은데

In [39]:
from sklearn.cluster import KMeans
# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters = 3, max_iter = 10000, random_state = 0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

# 소속 클러스터를 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

filename  \
0        battery-life_ipod_nano_8gb   
1             screen_netbook_1005ha   
2           keyboard_netbook_1005ha   
3       battery-life_netbook_1005ha   
4    satellite_garmin_nuvi_255W_gps   
5     accuracy_garmin_nuvi_255W_gps   
6       screen_garmin_nuvi_255W_gps   
7        speed_garmin_nuvi_255W_gps   
8      updates_garmin_nuvi_255W_gps   
9      display_garmin_nuvi_255W_gps   
11         size_asus_netbook_1005ha   
12              video_ipod_nano_8gb   
14                features_windows7   
13             screen_ipod_nano_8gb   
23         navigation_amazon_kindle   
16                   speed_windows7   
30              fonts_amazon_kindle   
27       performance_netbook_1005ha   
34       battery-life_amazon_kindle   
29         price_holiday_inn_london   
22      location_holiday_inn_london   
26          food_holiday_inn_london   
21       service_holiday_inn_london   
28           food_swissotel_chicago   
31  service_swissotel_hotel_chicago   
24          staff_swissotel_chicago   
46   location_bestwestern_hotel_sfo   
39   bathroom_bestwestern_hotel_sfo   
36          rooms_swissotel_chicago   
37    service_bestwestern_hotel_sfo   
38    parking_bestwestern_hotel_sfo   
18          room_holiday_inn_london   
49      staff_bestwestern_hotel_sfo   
40      rooms_bestwestern_hotel_sfo   
41       free_bestwestern_hotel_sfo   
17  directions_garmin_nuvi_255W_gps   
10              sound_ipod_nano_8gb   
15       voice_garmin_nuvi_255W_gps   
19            buttons_amazon_kindle   
20              price_amazon_kindle   
25       interior_toyota_camry_2007   
33    gas_mileage_toyota_camry_2007   
35    eyesight-issues_amazon_kindle   
32        comfort_toyota_camry_2007   
43       interior_honda_accord_2008   
44   transmission_toyota_camry_2007   
45        comfort_honda_accord_2008   
42    performance_honda_accord_2008   
47        quality_toyota_camry_2007   
48          seats_honda_accord_2008   
50        mileage_honda_accord_2008   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   opinion_text  \
0                                                                                                                                                                                                                                                                                                  short battery life  I moved up from an 8gb .\n0                                                                                                                                                                                                                                                                                            I love this ipod except for the battery life .\n1                             ...   
1                                                                                                                                                                                                                                                                           Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0                                                                                                                                                                                                       

### 군집별 핵심 단어 추출하기

KMeans 객체

각 군집을 구성하는 단어 피러가 군집의 중심(Centroid)을 기준으로 얼마나 가깝게 위치해 있는지 clusters_centers_라는 속성으로 제공

배열 값으로 제공 :  행은 개별 군집, 열은 개별 피처 의미 → 각 배열 내의 값은 개별 군집 내의 상대 위치를 숫자 값으로 표현한 일종의 좌표 값

In [40]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 5648)
[[0.00225177 0.00166569 0.         ... 0.         0.         0.        ]
 [0.         0.00043948 0.00099222 ... 0.00197    0.00156757 0.00156757]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [41]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}

    # cluster.centers array의 값이 큰 순으로 정렬된 인덱스 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:, ::-1]

    # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster']=cluster_num

        # cluster_centers_.argsort()[:,::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]

        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심 위치 상댓값, 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()

        cluster_details[cluster_num]['filenames'] = filenames

    return cluster_details

In [42]:
# print_cluster_details() 함수 생성
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('===========================')

In [43]:
feature_names = tfidf_vect.get_feature_names_out()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df, feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'video', 'size', 'feature', 'speed', 'map']
Reviews 파일명 : ['battery-life_ipod_nano_8gb', 'screen_netbook_1005ha', 'keyboard_netbook_1005ha', 'battery-life_netbook_1005ha', 'satellite_garmin_nuvi_255W_gps', 'accuracy_garmin_nuvi_255W_gps', 'screen_garmin_nuvi_255W_gps']
####### Cluster 1
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'price', 'parking', 'clean']
Reviews 파일명 : ['room_holiday_inn_london', 'service_holiday_inn_london', 'location_holiday_inn_london', 'staff_swissotel_chicago', 'food_holiday_inn_london', 'food_swissotel_chicago', 'price_holiday_inn_london']
####### Cluster 2
Top features: ['interior', 'seat', 'mileage', 'gas', 'comfortable', 'gas mileage', 'quality', 'car', 'transmission', 'voice']
Reviews 파일명 : ['sound_ipod_nano_8gb', 'voice_garmin_nuvi_255W_gps', 'directions_garmin_nuvi_255W_gps', 'buttons_amazon_kindle', 'price_amazon_kindle', 'in